In [ ]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

In [ ]:
code = pd.read_excel(r"/content/totalmerged.xlsx")
code

,Question,Correct_Code,Code_with_Error,Total_Marks
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0
...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0


In [ ]:
# Merge columns
code['merged'] = code['Question'] + ' ' + code['Code_with_Error']
code

,Question,Correct_Code,Code_with_Error,Total_Marks,merged
0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,Print the factors of a number #include <stdio....
1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,Print the factors of a number #include <stdio....
2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,Print the factors of a number #include <stdio....
3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,Print the factors of a number #include <stdio....
4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,Print the factors of a number #include <stdio....
...,...,...,...,...,...
995,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define MAX_SIZE 100 \n...,7.0,Print maximum Occurring Character in a String ...
996,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
997,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\nchar getMaxOccurringCha...,6.0,Print maximum Occurring Character in a String ...
998,Print maximum Occurring Character in a String ...,#include <stdio.h>\n\n#define ASCII_SIZE 128\n...,#include <stdio.h> \n\n#define ASCII_SIZE 128 ...,5.0,Print maximum Occurring Character in a String ...


In [ ]:
# Fill missing values with empty strings
code['merged'] = code['merged'].fillna('')

In [ ]:
# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
# Tokenize and convert the textual data to vectors
X_text = code["merged"].values
y = code["Total_Marks"].values

In [ ]:
X_vectors = []
for text in X_text:
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='tf', max_length=512, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    output = roberta_model(input_ids, attention_mask=attention_mask)[0][:, 0, :]
    X_vectors.append(output.numpy())

In [ ]:

X_vectors = np.array(X_vectors)
y = np.array(y)

In [ ]:
X_vectors

array([[[-0.04381751,  0.01298395, -0.06823917, ..., -0.01078259,
         -0.06515283,  0.00736105]],

       [[-0.03485741,  0.02759037, -0.0669408 , ...,  0.01000887,
         -0.06539918,  0.01887213]],

       [[-0.04260369,  0.00817573, -0.0670611 , ..., -0.01272669,
         -0.06602035,  0.00926103]],

       ...,

       [[-0.02076413,  0.02080108, -0.06215055, ..., -0.03736269,
         -0.05828972,  0.02422302]],

       [[-0.00893429,  0.01403939, -0.07219779, ..., -0.02958895,
         -0.0384867 ,  0.01692103]],

       [[-0.01983733,  0.01483572, -0.06987616, ..., -0.03227926,
         -0.05074834,  0.02020183]]], dtype=float32)

In [ ]:
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [ ]:
# Create the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')


In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[early_stopping])

Epoch 1/50
9/9 [==============================] - 0s 27ms/step - loss: 4.4392 - val_loss: 5.3367
Epoch 2/50
9/9 [==============================] - 0s 19ms/step - loss: 4.2852 - val_loss: 5.3104
Epoch 3/50
9/9 [==============================] - 0s 20ms/step - loss: 4.6058 - val_loss: 5.2253
Epoch 4/50
9/9 [==============================] - 0s 21ms/step - loss: 4.4236 - val_loss: 5.1936
Epoch 5/50
9/9 [==============================] - 0s 19ms/step - loss: 4.4240 - val_loss: 5.2030
Epoch 6/50
9/9 [==============================] - 0s 20ms/step - loss: 4.3508 - val_loss: 5.1949
Epoch 7/50
9/9 [==============================] - 0s 19ms/step - loss: 4.2858 - val_loss: 5.1878
Epoch 8/50
9/9 [==============================] - 0s 19ms/step - loss: 4.4024 - val_loss: 5.1059
Epoch 9/50
9/9 [==============================] - 0s 20ms/step - loss: 4.3588 - val_loss: 5.0771
Epoch 10/50
9/9 [==============================] - 0s 21ms/step - loss: 4.1960 - val_loss: 5.0901
Epoch 11/50
9/9 [============

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


8/8 [==============================] - 0s 5ms/step


In [ ]:
# Calculate evaluation metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [ ]:
# Print the evaluation metrics
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print() 
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)
print() 
print("Train MAPE:", train_mape)
print("Test MAPE:", test_mape)
print() 
print("Train R^2:", train_r2)
print("Test R^2:", test_r2)

Train RMSE: 2.0840238739886017
Test RMSE: 2.192497845908926

Train MAE: 1.6558579601002759
Test MAE: 1.7370490894317627

Train MAPE: 95916035021653.16
Test MAPE: 331366081325171.06

Train R^2: 0.10303816805624522
Test R^2: 0.09377673600009351


In [ ]:
!pip install pyforest
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=7f6a80f02c602dd6b9ebd0e131546fb49c71c4ca3fc82574ffc844b9656ed277
  Stored in directory: /root/.cache/pip/wheels/9e/7d/2c/5d2f5e62de376c386fd3bf5a8e5bd119ace6a9f48f49df6017
Successfully built pyforest
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing important libraries
import pyforest
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix
# Scikit-learn packages
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
# Hide warnings
import warnings
warnings.filterwarnings("ignore")
# Setting up max columns displayed to 100
pd.options.display.max_columns = 100

In [ ]:
# Reshape the X_train and X_test arrays
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

In [ ]:
reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train_reshaped, X_test_reshaped, y_train, y_test)
print(models)

 21%|██▏       | 9/42 [01:38<07:50, 14.25s/it]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


 57%|█████▋    | 24/42 [03:40<01:48,  6.05s/it]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 76%|███████▌  | 32/42 [03:56<00:35,  3.52s/it]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 562.


100%|██████████| 42/42 [04:38<00:00,  6.64s/it]

                               Adjusted R-Squared       R-Squared      RMSE  \
Model                                                                         
SGDRegressor                       15935008286.95 -33213932932.37 419741.32   
Lars                                     75215.29      -156770.95    911.92   
TransformedTargetRegressor                  12.03          -21.99     11.04   
LinearRegression                            12.03          -21.99     11.04   
KernelRidge                                  5.46           -8.29      7.02   
GaussianProcessRegressor                     4.50           -6.29      6.22   
Ridge                                        1.68           -0.43      2.75   
LinearSVR                                    1.64           -0.33      2.66   
ExtraTreeRegressor                           1.60           -0.25      2.58   
DecisionTreeRegressor                        1.58           -0.21      2.53   
PassiveAggressiveRegressor                   1.57   

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesRegressor

# Random Forest Regressor
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X_train_reshaped, y_train)
y_train_pred_rf = rf_regressor.predict(X_train_reshaped)
y_test_pred_rf = rf_regressor.predict(X_test_reshaped)

# Gradient Boosting Regressor
gra_regressor = GradientBoostingRegressor()
gra_regressor.fit(X_train_reshaped, y_train)
y_train_pred_gra = gra_regressor.predict(X_train_reshaped)
y_test_pred_gra = gra_regressor.predict(X_test_reshaped)

# ExtraTreesRegressor
et_regressor = ExtraTreesRegressor()
et_regressor.fit(X_train_reshaped, y_train)
y_train_pred_et = et_regressor.predict(X_train_reshaped)
y_test_pred_et = et_regressor.predict(X_test_reshaped)

# CatBoost Regressor
catboost_regressor = CatBoostRegressor()
catboost_regressor.fit(X_train_reshaped, y_train)
y_train_pred_catboost = catboost_regressor.predict(X_train_reshaped)
y_test_pred_catboost = catboost_regressor.predict(X_test_reshaped)

# LGBM Regressor
xgb_regressor = lgb.LGBMRegressor()
xgb_regressor.fit(X_train_reshaped, y_train)
y_train_pred_xgb = xgb_regressor.predict(X_train_reshaped)
y_test_pred_xgb = xgb_regressor.predict(X_test_reshaped)

Learning rate set to 0.03738
0:	learn: 2.1841532	total: 394ms	remaining: 6m 33s
1:	learn: 2.1639475	total: 585ms	remaining: 4m 51s
2:	learn: 2.1453095	total: 777ms	remaining: 4m 18s
3:	learn: 2.1284584	total: 961ms	remaining: 3m 59s
4:	learn: 2.1118236	total: 1.14s	remaining: 3m 47s
5:	learn: 2.0974882	total: 1.34s	remaining: 3m 41s
6:	learn: 2.0853071	total: 1.53s	remaining: 3m 37s
7:	learn: 2.0672525	total: 1.72s	remaining: 3m 33s
8:	learn: 2.0548193	total: 1.92s	remaining: 3m 31s
9:	learn: 2.0419752	total: 2.1s	remaining: 3m 28s
10:	learn: 2.0313450	total: 2.29s	remaining: 3m 26s
11:	learn: 2.0165697	total: 2.49s	remaining: 3m 24s
12:	learn: 2.0063117	total: 2.68s	remaining: 3m 23s
13:	learn: 1.9919981	total: 2.87s	remaining: 3m 21s
14:	learn: 1.9785031	total: 3.05s	remaining: 3m 20s
15:	learn: 1.9660864	total: 3.23s	remaining: 3m 18s
16:	learn: 1.9516961	total: 3.43s	remaining: 3m 18s
17:	learn: 1.9415135	total: 3.62s	remaining: 3m 17s
18:	learn: 1.9268139	total: 3.81s	remaining: 3

In [ ]:
# Calculate evaluation metrics for Random Forest Regressor
mape_train_rf = mean_absolute_percentage_error(y_train, y_train_pred_rf)
mape_test_rf = mean_absolute_percentage_error(y_test, y_test_pred_rf)
r2_train_rf = r2_score(y_train, y_train_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)
rmse_train_rf = mean_squared_error(y_train, y_train_pred_rf, squared=False)
rmse_test_rf = mean_squared_error(y_test, y_test_pred_rf, squared=False)
mae_train_rf = mean_absolute_error(y_train, y_train_pred_rf)
mae_test_rf = mean_absolute_error(y_test, y_test_pred_rf)

# Calculate evaluation metrics for CatBoost Regressor
mape_train_catboost = mean_absolute_percentage_error(y_train, y_train_pred_catboost)
mape_test_catboost = mean_absolute_percentage_error(y_test, y_test_pred_catboost)
r2_train_catboost = r2_score(y_train, y_train_pred_catboost)
r2_test_catboost = r2_score(y_test, y_test_pred_catboost)
rmse_train_catboost = mean_squared_error(y_train, y_train_pred_catboost, squared=False)
rmse_test_catboost = mean_squared_error(y_test, y_test_pred_catboost, squared=False)
mae_train_catboost = mean_absolute_error(y_train, y_train_pred_catboost)
mae_test_catboost = mean_absolute_error(y_test, y_test_pred_catboost)

# Calculate evaluation metrics for XGBoost Regressor
mape_train_xgb = mean_absolute_percentage_error(y_train, y_train_pred_xgb)
mape_test_xgb = mean_absolute_percentage_error(y_test, y_test_pred_xgb)
r2_train_xgb = r2_score(y_train, y_train_pred_xgb)
r2_test_xgb = r2_score(y_test, y_test_pred_xgb)
rmse_train_xgb = mean_squared_error(y_train, y_train_pred_xgb, squared=False)
rmse_test_xgb = mean_squared_error(y_test, y_test_pred_xgb, squared=False)
mae_train_xgb = mean_absolute_error(y_train, y_train_pred_xgb)
mae_test_xgb = mean_absolute_error(y_test, y_test_pred_xgb)

# Calculate evaluation metrics for Gradient Boosting Regressor
mape_train_gra = mean_absolute_percentage_error(y_train, y_train_pred_gra)
mape_test_gra = mean_absolute_percentage_error(y_test, y_test_pred_gra)
r2_train_gra = r2_score(y_train, y_train_pred_gra)
r2_test_gra = r2_score(y_test, y_test_pred_gra)
rmse_train_gra = mean_squared_error(y_train, y_train_pred_gra, squared=False)
rmse_test_gra = mean_squared_error(y_test, y_test_pred_gra, squared=False)
mae_train_gra = mean_absolute_error(y_train, y_train_pred_gra)
mae_test_gra = mean_absolute_error(y_test, y_test_pred_gra)

# Calculate evaluation metrics for ExtraTreesRegressor
mape_train_et = mean_absolute_percentage_error(y_train, y_train_pred_et)
mape_test_et = mean_absolute_percentage_error(y_test, y_test_pred_et)
r2_train_et = r2_score(y_train, y_train_pred_et)
r2_test_et = r2_score(y_test, y_test_pred_et)
rmse_train_et = mean_squared_error(y_train, y_train_pred_et, squared=False)
rmse_test_et = mean_squared_error(y_test, y_test_pred_et, squared=False)
mae_train_et = mean_absolute_error(y_train, y_train_pred_et)
mae_test_et = mean_absolute_error(y_test, y_test_pred_et)


In [ ]:
# Print the evaluation metrics for Random Forest Regressor
print("Random Forest Regressor:")
print("Train RMSE:", rmse_train_rf)
print("Test RMSE:", rmse_test_rf)
print("Train MAE:", mae_train_rf)
print("Test MAE:", mae_test_rf)
print("Train MAPE:", mape_train_rf)
print("Test MAPE:", mape_test_rf)
print("Train R^2:", r2_train_rf)
print("Test R^2:", r2_test_rf)

# Print the evaluation metrics for CatBoost Regressor
print("CatBoost Regressor:")
print("Train RMSE:", rmse_train_catboost)
print("Test RMSE:", rmse_test_catboost)
print("Train MAE:", mae_train_catboost)
print("Test MAE:", mae_test_catboost)
print("Train MAPE:", mape_train_catboost)
print("Test MAPE:", mape_test_catboost)
print("Train R^2:", r2_train_catboost)
print("Test R^2:", r2_test_catboost)

# Print the evaluation metrics for XGBoost Regressor
print("XGBoost Regressor:")
print("Train RMSE:", rmse_train_xgb)
print("Test RMSE:", rmse_test_xgb)
print("Train MAE:", mae_train_xgb)
print("Test MAE:", mae_test_xgb)
print("Train MAPE:", mape_train_xgb)
print("Test MAPE:", mape_test_xgb)
print("Train R^2:", r2_train_xgb)
print("Test R^2:", r2_test_xgb)

# Print the evaluation metrics for Gradient Boosting Regressor
print("Gradient Boosting Regressor:")
print("Train RMSE:", rmse_train_gra)
print("Test RMSE:", rmse_test_gra)
print("Train MAE:", mae_train_gra)
print("Test MAE:", mae_test_gra)
print("Train MAPE:", mape_train_gra)
print("Test MAPE:", mape_test_gra)
print("Train R^2:", r2_train_gra)
print("Test R^2:", r2_test_gra)

# Print the evaluation metrics for ExtraTreesRegressor
print("ExtraTreesRegressor:")
print("Train RMSE:", rmse_train_et)
print("Test RMSE:", rmse_test_et)
print("Train MAE:", mae_train_et)
print("Test MAE:", mae_test_et)
print("Train MAPE:", mape_train_et)
print("Test MAPE:", mape_test_et)
print("Train R^2:", r2_train_et)
print("Test R^2:", r2_test_et)

Random Forest Regressor:
Train RMSE: 0.8374304594379701
Test RMSE: 1.9061985032996662
Train MAE: 0.5580601102070508
Test MAE: 1.4180283241449791
Train MAPE: 32855442121386.336
Test MAPE: 248345101013331.9
Train R^2: 0.8551674791061156
Test R^2: 0.3149960045158233
CatBoost Regressor:
Train RMSE: 0.5654142536136214
Test RMSE: 1.8544947589027168
Train MAE: 0.2645235303948474
Test MAE: 1.3989908672800966
Train MAPE: 1881030442500.7712
Test MAPE: 198388633187903.3
Train R^2: 0.9339759610312499
Test R^2: 0.35165214735351347
XGBoost Regressor:
Train RMSE: 0.5535738731566742
Test RMSE: 1.8919196539096232
Train MAE: 0.19780915230290186
Test MAE: 1.415141092654428
Train MAPE: 15296979052016.682
Test MAPE: 185488687965413.4
Train R^2: 0.9367122358847497
Test R^2: 0.32521995035718676
Gradient Boosting Regressor:
Train RMSE: 0.9094385494306155
Test RMSE: 1.9040652086431076
Train MAE: 0.6789543219840515
Test MAE: 1.459500429968728
Train MAPE: 12978525246082.219
Test MAPE: 194110411017023.0
Train R^2